# EDA and Cleaning

In the last notebook I have collected the data in this I will explolre the data and clean it.

---

In [2]:
# Imports
import pandas as pd

In [3]:
# Get the data
reddit = pd.read_csv('../data/reddit.csv')

In [4]:
reddit.head()

,subreddit,selftext,title,created_utc
0,bioinformatics,Hi \nSo I was reading an article about finding...,why child data are less than monozygotic twin(...,1673029838
1,bioinformatics,"Hello everyone, i’m a biotechnology student fr...",Masters in bioinformatic Europe,1673019129
2,bioinformatics,&amp;#x200B;\n\nhttps://preview.redd.it/adr3qi...,Help interpreting a PCA plot,1673013992
3,bioinformatics,NaN,Help interpreting a PCA plot,1673013802
4,bioinformatics,"Hi all,\n\nI am planning on running a microarr...",Microarray metanalysis using raw data vs serie...,1673007061


In [5]:
# Check for missing data
reddit.isnull().sum()

subreddit        0
selftext       202
title            0
created_utc      0
dtype: int64

In my subreddits the text of the post is precent in most of the cases, however, I will model on title first and then add text and see if accuracy will improve. The created_utc was used for collecting the data and I do not neet it for modeling

In [6]:
reddit.drop(columns = 'created_utc', inplace = True)

In [7]:
# I have almost the same number of posts for Biotech and Bioinformatics. The base line should be close to 50%
reddit['subreddit'].value_counts(normalize=True)

biotech           0.539683
bioinformatics    0.451346
u_uni-biotech     0.008972
Name: subreddit, dtype: float64

Some of Biotech subreddit have different names. I need to change them to Biotech

In [8]:
reddit['subreddit'] = reddit['subreddit'].map(lambda x: 'biotech' if x != 'bioinformatics' else 'bioinformatics')

In [9]:
# Check the base line
reddit['subreddit'].value_counts(normalize=True)

biotech           0.548654
bioinformatics    0.451346
Name: subreddit, dtype: float64

In [10]:
# Change titles to lower case
reddit['title'] = reddit['title'].str.lower()
reddit['selftext'] = reddit['selftext'].str.lower()

In [11]:
reddit.head()

,subreddit,selftext,title
0,bioinformatics,hi \nso i was reading an article about finding...,why child data are less than monozygotic twin(...
1,bioinformatics,"hello everyone, i’m a biotechnology student fr...",masters in bioinformatic europe
2,bioinformatics,&amp;#x200b;\n\nhttps://preview.redd.it/adr3qi...,help interpreting a pca plot
3,bioinformatics,NaN,help interpreting a pca plot
4,bioinformatics,"hi all,\n\ni am planning on running a microarr...",microarray metanalysis using raw data vs serie...


In [12]:
# Change the subreddit names to numbers for modeling
reddit['subreddit_num'] = reddit['subreddit'].map({'bioinformatics': 0, 'biotech': 1})
reddit.head()

,subreddit,selftext,title,subreddit_num
0,bioinformatics,hi \nso i was reading an article about finding...,why child data are less than monozygotic twin(...,0
1,bioinformatics,"hello everyone, i’m a biotechnology student fr...",masters in bioinformatic europe,0
2,bioinformatics,&amp;#x200b;\n\nhttps://preview.redd.it/adr3qi...,help interpreting a pca plot,0
3,bioinformatics,NaN,help interpreting a pca plot,0
4,bioinformatics,"hi all,\n\ni am planning on running a microarr...",microarray metanalysis using raw data vs serie...,0
